In [4]:
# === CAMS CO₂ 2020 Interactive Globe (統合版) ===
import numpy as np
import xarray as xr
import plotly.graph_objects as go
import geopandas as gpd
import requests, zipfile, io

# --- 1) データ読込（自動変数＋軸検出） ---
url = "https://raw.githubusercontent.com/philhawk-cyber/era5-cams-visuals/main/data/fd9c5180844360480e5575ed69dc8799.nc"
ds = xr.open_dataset(url)
print(ds)

# 利用可能な変数と座標を表示
print("📦 Variables:", list(ds.data_vars))
print("🧭 Coordinates:", list(ds.coords))

# --- CO₂変数を検出 ---
for varname in ["co2", "xco2", "tcco2"]:
    if varname in ds.data_vars:
        co2 = ds[varname]
        print(f"✅ Using variable: '{varname}'")
        break
else:
    raise KeyError("❌ No CO₂-related variable found (expected 'co2', 'xco2', or 'tcco2').")

# --- 軸名の違いに対応 ---
time_key = "time" if "time" in ds.coords else "valid_time"
lat_key = "latitude" if "latitude" in ds.coords else "lat"
lon_key = "longitude" if "longitude" in ds.coords else "lon"

times = ds[time_key]
lats = ds[lat_key]
lons = ds[lon_key]

print(f"✅ Data shape: {co2.shape}")
print(f"🧭 Time range: {str(times.values[0])[:10]} → {str(times.values[-1])[:10]}")
print(f"📊 Lat: {lats.values.min()}–{lats.values.max()}  Lon: {lons.values.min()}–{lons.values.max()}")

# --- 2) 球面座標に変換 ---
lon_grid, lat_grid = np.meshgrid(lons, lats)
X = np.cos(np.deg2rad(lat_grid)) * np.cos(np.deg2rad(lon_grid))
Y = np.cos(np.deg2rad(lat_grid)) * np.sin(np.deg2rad(lon_grid))
Z = np.sin(np.deg2rad(lat_grid))

# --- 3) 経度の閉鎖処理（縦線除去・ブレンド法） ---
lons = co2.longitude.values

# 経度360°が欠けている場合は追加
if not np.isclose(lons[-1], 360.0, atol=0.5):
    extra_slice = co2.isel(longitude=0).copy(deep=True)
    co2 = xr.concat([co2, extra_slice], dim="longitude")
    lons_new = np.linspace(0, 360, co2.sizes["longitude"])
    co2 = co2.assign_coords(longitude=lons_new)

# 経度0°と360°を平均して滑らかに接続（縦線除去の最終対策）
co2_vals = co2.values
co2_vals[..., -1] = (co2_vals[..., -2] + co2_vals[..., 0]) / 2
co2[:] = co2_vals

# 経度を四捨五入
co2["longitude"] = np.round(co2["longitude"], 4)

# --- 4) 海岸線データ取得（GitHubミラー使用） ---
url = "https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_110m_admin_0_countries.geojson"
world = gpd.read_file(url)

coast_x, coast_y = [], []
for geom in world.geometry:
    if geom.geom_type == "Polygon":
        x, y = geom.exterior.xy
        coast_x += list(x) + [None]
        coast_y += list(y) + [None]
    elif geom.geom_type == "MultiPolygon":
        for poly in geom.geoms:
            x, y = poly.exterior.xy
            coast_x += list(x) + [None]
            coast_y += list(y) + [None]

# None 除去
coast_x_valid = np.array([x for x in coast_x if x is not None])
coast_y_valid = np.array([y for y in coast_y if y is not None])

# 球面変換
coast_X = np.cos(np.deg2rad(coast_y_valid)) * np.cos(np.deg2rad(coast_x_valid))
coast_Y = np.cos(np.deg2rad(coast_y_valid)) * np.sin(np.deg2rad(coast_x_valid))
coast_Z = np.sin(np.deg2rad(coast_y_valid))

coast_trace = go.Scatter3d(
    x=coast_X,
    y=coast_Y,
    z=coast_Z,
    mode="lines",
    line=dict(color="black", width=0.8),
    showlegend=False
)

# --- 5) カラーと照明設定 ---
vmin, vmax = np.nanpercentile(co2.values, [2, 98])
colorscale = "Turbo"  # 鮮やか、または "RdYlGn_r"

def make_surface(month_idx):
    # 時間軸を自動判別して抽出（time または valid_time）
    co2_frame = co2.isel({time_key: month_idx}).values

    return go.Surface(
        x=X, y=Y, z=Z,
        surfacecolor=co2_frame,
        colorscale=colorscale,
        cmin=vmin, cmax=vmax,
        showscale=False,
        lighting=dict(
            ambient=1.0, diffuse=0.0, specular=0.0,
            roughness=1.0, fresnel=0.0
        ),
        opacity=1.0
    )

# --- 6) フレーム生成 ---
frames = [
    go.Frame(name=f"Month {i+1}", data=[make_surface(i), coast_trace])
    for i in range(co2.sizes[time_key])  # ← 修正ポイント: len(times) → co2.sizes[time_key]
]

# --- 7) 図作成 ---
fig = go.Figure(
    data=[make_surface(0), coast_trace],
    frames=frames
)

fig.update_layout(
    title="🌍 CAMS Global CO₂ Distribution (2020)",
    width=1100, height=750,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        aspectmode="data",
        bgcolor="white"
    ),
    margin=dict(l=40, r=40, t=60, b=20),
    updatemenus=[dict(
        type="buttons",
        x=0.01, y=0.01, xanchor="left", yanchor="bottom",
        buttons=[
            dict(label="▶ Play", method="animate",
                 args=[None, {"frame": {"duration": 700, "redraw": True},
                              "fromcurrent": True, "transition": {"duration": 0}}]),
            dict(label="⏸ Pause", method="animate",
                 args=[[None], {"mode": "immediate",
                                "frame": {"duration": 0, "redraw": False},
                                "transition": {"duration": 0}}]),
        ]
    )],
    sliders=[dict(
        active=0,
        x=0.12, y=0.04, xanchor="left", yanchor="bottom",
        len=0.76,
        currentvalue=dict(prefix="Month "),
        steps=[dict(method="animate",
                    args=[[f"Month {i+1}"],
                          {"mode": "immediate",
                           "frame": {"duration": 0, "redraw": True},
                           "transition": {"duration": 0}}],
                    label=f"{i+1:02d}") for i in range(len(times))]
    )]
)

# --- 8) 出典ラベルとカラーバー ---
fig.add_annotation(
    text="Source: Copernicus Atmosphere Monitoring Service (CAMS), ECMWF",
    xref="paper", yref="paper",
    x=0, y=-0.05, showarrow=False, font=dict(size=10, color="gray")
)

fig.update_layout(coloraxis_colorbar=dict(
    title="CO₂ mol/mol × 1e6",
    y=0.5, len=0.9
))

fig.show()


Output hidden; open in https://colab.research.google.com to view.